In [ ]:
'''聯電'''
import pandas as pd
from itertools import chain
from collections import defaultdict
import re
import pickle    
    
def preprocess(classorall, df, class_index, n, stopword): 
    class_TFdict = dict()
    class_DFdict = dict()
    
    for index, row in df.iloc[class_index].iterrows(): 
        text = str(row["title"]) + str(row["content"])  
        text = re.sub(r'[^\w]','', text)  #remove string not unicode decoded
        text = re.sub(r"[A-Za-z0-9]", "", text) #remove english & number

        doc_has_word = []
        for t in range(len(text)-(n-1)):
            words = text[t:t+n]  
            # tf, df
            if words not in stopword:
                if words in class_TFdict:      #term freq of that class
                    class_TFdict[words] += 1
                else:
                    class_TFdict[words] = 1
                if words not in doc_has_word:
                    if words in class_DFdict:     #df
                        class_DFdict[words] += 1
                    else:
                        class_DFdict[words] = 1  
                    doc_has_word.append(words)
                
    if classorall == 'class':
        key_to_remove = []
        for key, val in class_DFdict.items():  #remove the terms whose df <= 10
            if val <= 10:
                key_to_remove.append(key)
        for key in key_to_remove:
            del class_DFdict[key]
            del class_TFdict[key]
        
    return class_TFdict, class_DFdict


df1 = pd.read_csv('../dataset/utf8/news.csv', encoding="UTF-8")[['post_time','title','content']]
df2 = pd.read_csv('../dataset/utf8/forum.csv', encoding="UTF-8")[['post_time','title','content']]
df3 = pd.read_csv('../dataset/utf8/bbs.csv', encoding="UTF-8")[['post_time','title','content']]

stopword_list = []
with open('../dataset/stopwords.txt', 'r', encoding='UTF-8') as file:
    for data in file.readlines():
        data = data.strip()    
        stopword_list.append(data)
        
risedown =[ ['慘綠','跌','下挫','利空', '看衰','減碼','綠盤','低點','收綠','收黑','重挫','賣超','新低','摜破','賠','虧','黑天鵝','空方','停工','信用破產','悲觀','斷頭','損失'] , ['漲','利多','布局','看好','成長','加碼','上升','上看','紅盤','收紅','買超','攀升','高點','高峰','新高','降息','多方','上車','樂觀', '護盤','台股漲']]

df = pd.concat([df1,df2,df3],axis=0)  
all_doc_count = df.shape[0]
all_doc = list(range(all_doc_count)) 

In [ ]:
ngrm = [2,3,4,5,6]
# 所有文章
all_ngram_count = []    
all_ngrm = [] 
for i in ngrm:
    print('all ', i)
    all_TF, all_DF = preprocess('all', df, all_doc, i, stopword_list) 
    all_ngram_count.append(len(all_TF))         
    # merge (ngram, tf, df)
    ngram_tf_df = defaultdict(list)
    for k, v in chain(all_TF.items(), all_DF.items()):
        ngram_tf_df[k].append(v)   
    file = open('all_ngrm[{}].pickle'.format(i), 'wb')
    pickle.dump(list(ngram_tf_df.items()), file)
    file.close()        

all  2
all  5
all  6
